In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from vecstack import stacking

import optuna

In [2]:
train = pd.read_csv('preprocessed_train.csv', encoding='euc-kr')
test = pd.read_csv('preprocessed_test.csv', encoding='euc-kr')

In [3]:
train

,분석데이터,label,numstrings,avlength,printables,entropy,paths,urls,registry,MZ,...,dist_86,dist_87,dist_88,dist_89,dist_90,dist_91,dist_92,dist_93,dist_94,dist_95
0,1,1,144,12.298611,1771,5.356616,0,0,0,1,...,10,4,10,9,4,0,1,0,0,0
1,2,1,804,9.580846,7703,6.063542,0,0,0,6,...,43,121,84,78,47,36,40,45,27,36
2,3,0,2205,12.736054,28083,6.107050,9,0,0,6,...,326,268,239,286,199,148,154,37,48,36
3,4,0,2602,10.288240,26770,5.373013,8,0,0,1,...,336,230,206,245,76,0,26,702,1,5
4,5,1,8980,23.252339,208806,5.775223,0,28,16,3,...,731,882,1171,1010,322,64,327,84,75,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,1,2018,13.938057,28127,5.940442,0,70,0,11,...,246,186,206,235,88,33,81,58,61,72
9996,9997,0,1105,16.437104,18163,5.766962,0,11,0,3,...,199,57,134,123,20,25,28,25,41,13
9997,9998,0,4,58.500000,234,3.811827,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9998,9999,1,3312,24.939312,82599,5.834730,0,39,0,8,...,438,985,806,851,113,123,181,100,75,86


In [4]:
x_train = train.drop(columns=['분석데이터', 'label'], axis=1)

In [5]:
y_train = train['label']

In [6]:
x_test = test.drop(columns='분석대상', axis=1)

In [7]:
# train_df_x = train.drop(['label'], axis=1)

In [8]:
# train_df_y = train['label']

In [9]:
# x_train, x_test, y_train, y_test = train_test_split(train_df_x, train_df_y, test_size=0.2, random_state=42)

In [10]:
"""
from xgboost import XGBClassifier

xgb_final = XGBClassifier(silent=False,
                              n_estimators = 10000,
                              booster='gbtree',
                              tree_method='gpu_hist',
                              preidctor= 'gpu_predictor',
                              scale_pos_weight=1,
                              learning_rate=0.03689407512484644,
                              objective='binary:logistic',
                              max_depth = 8,
                              subsample = 0.780714581166012,
                              colsample_bytree = 0.3723914688159835,
                              gamma = 0,
                              reg_lambda = 50.0,
                              random_state=42)
"""

"\nfrom xgboost import XGBClassifier\n\nxgb_final = XGBClassifier(silent=False,\n                              n_estimators = 10000,\n                              booster='gbtree',\n                              tree_method='gpu_hist',\n                              preidctor= 'gpu_predictor',\n                              scale_pos_weight=1,\n                              learning_rate=0.03689407512484644,\n                              objective='binary:logistic',\n                              max_depth = 8,\n                              subsample = 0.780714581166012,\n                              colsample_bytree = 0.3723914688159835,\n                              gamma = 0,\n                              reg_lambda = 50.0,\n                              random_state=42)\n"

In [11]:
# final_pred = xgb_final.fit(x_train, y_train).predict(x_test)

In [12]:
# accuracy_score(y_test, final_pred)

In [13]:
# confusion_matrix(y_test, final_pred)

In [14]:
"""
cb_model = CatBoostClassifier(objective = 'CrossEntropy',
                              colsample_bylevel = 0.09901213513123053,
                              depth = 11,
                              boosting_type = 'Ordered',
                              bootstrap_type = 'MVS',
                              learning_rate = 0.11016052222039235,
                              iterations = 175)
"""

"\ncb_model = CatBoostClassifier(objective = 'CrossEntropy',\n                              colsample_bylevel = 0.09901213513123053,\n                              depth = 11,\n                              boosting_type = 'Ordered',\n                              bootstrap_type = 'MVS',\n                              learning_rate = 0.11016052222039235,\n                              iterations = 175)\n"

In [15]:
# cb_pred = cb_model.fit(x_train, y_train).predict(x_test)

In [16]:
# accuracy_score(y_test, cb_pred)

In [17]:
# confusion_matrix(y_test, cb_pred)

In [18]:
"""
lgbm_model = LGBMClassifier(n_estimators = 10000,
                            learning_rate = 0.09416659111369403,
                            max_depth = 43,
                            boosting = 'gbdt',
                            objective = 'binary',
                            metric = 'binary_logloss',
                            is_training_metric = True,
                            num_leaves = 41,
                            min_data_in_leaf = 10,
                            feature_fraction = 0.8,
                            bagging_fraction = 0.9,
                            bagging_freq = 0,
                            alpha = 0.019782149081578264)
"""

"\nlgbm_model = LGBMClassifier(n_estimators = 10000,\n                            learning_rate = 0.09416659111369403,\n                            max_depth = 43,\n                            boosting = 'gbdt',\n                            objective = 'binary',\n                            metric = 'binary_logloss',\n                            is_training_metric = True,\n                            num_leaves = 41,\n                            min_data_in_leaf = 10,\n                            feature_fraction = 0.8,\n                            bagging_fraction = 0.9,\n                            bagging_freq = 0,\n                            alpha = 0.019782149081578264)\n"

In [19]:
# lgbm_pred = lgbm_model.fit(x_train, y_train).predict(x_test)

In [20]:
# accuracy_score(y_test, lgbm_pred)

In [21]:
# confusion_matrix(y_test, lgbm_pred)

In [22]:
xgb_params = {'n_estimators': 10000,
               'learning_rate': 0.03689407512484644,
               'max_depth': 8,
               'colsample_bytree': 0.3723914688159835,
               'subsample': 0.780714581166012,
               'eval_metric': 'auc',
               'use_label_encoder': False,
               'gamma': 0,
               'reg_lambda': 50.0,
               'tree_method': 'gpu_hist',
               'gpu_id': 0,
               'predictor': 'gpu_predictor',
               'random_state': 42 }

lgb_params = {'n_estimators': 10000,
              'learning_rate':0.09416659111369403,
              'max_depth':43,
              'boosting':'gbdt',
              'objective': 'binary',
              'metric': 'binary_logloss',
              'is_training_metric': True,
              'num_leaves':41,
              'min_data_in_leaf':10,
              'feature_fraction':0.8,
              'bagging_fraction':0.9,
              'bagging_freq':0,
              'alpha': 0.019782149081578264 }

cat_params = {'objective': 'CrossEntropy',
              'colsample_bylevel': 0.09901213513123053,
              'depth': 11,
              'boosting_type': 'Ordered',
              'bootstrap_type': 'MVS',
              'learning_rate': 0.11016052222039235,
              'iterations': 175
             }

In [23]:
lgbm = LGBMClassifier(**lgb_params)

xgb = XGBClassifier(**xgb_params)

cat = CatBoostClassifier(**cat_params)

In [24]:
models = [lgbm, xgb, cat]

In [25]:
S_train, S_test = stacking(models, x_train, y_train, x_test, regression=False,
                          metric=accuracy_score, n_folds=5, stratified=True, shuffle=True, random_state=42, verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMClassifier]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
    fold  0:  [0.93200000]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[Li

89:	learn: 0.1610161	total: 4m 9s	remaining: 3m 55s
90:	learn: 0.1598073	total: 4m 12s	remaining: 3m 53s
91:	learn: 0.1579638	total: 4m 15s	remaining: 3m 50s
92:	learn: 0.1573625	total: 4m 18s	remaining: 3m 47s
93:	learn: 0.1561509	total: 4m 20s	remaining: 3m 44s
94:	learn: 0.1547255	total: 4m 23s	remaining: 3m 41s
95:	learn: 0.1536903	total: 4m 26s	remaining: 3m 39s
96:	learn: 0.1527348	total: 4m 29s	remaining: 3m 36s
97:	learn: 0.1521888	total: 4m 32s	remaining: 3m 33s
98:	learn: 0.1499707	total: 4m 34s	remaining: 3m 30s
99:	learn: 0.1492879	total: 4m 37s	remaining: 3m 28s
100:	learn: 0.1486702	total: 4m 40s	remaining: 3m 25s
101:	learn: 0.1473527	total: 4m 42s	remaining: 3m 22s
102:	learn: 0.1461530	total: 4m 45s	remaining: 3m 19s
103:	learn: 0.1451575	total: 4m 48s	remaining: 3m 16s
104:	learn: 0.1438124	total: 4m 51s	remaining: 3m 14s
105:	learn: 0.1429016	total: 4m 54s	remaining: 3m 11s
106:	learn: 0.1423662	total: 4m 56s	remaining: 3m 8s
107:	learn: 0.1411510	total: 4m 59s	remai

70:	learn: 0.1753459	total: 3m 19s	remaining: 4m 51s
71:	learn: 0.1734203	total: 3m 21s	remaining: 4m 48s
72:	learn: 0.1717970	total: 3m 24s	remaining: 4m 46s
73:	learn: 0.1708744	total: 3m 27s	remaining: 4m 42s
74:	learn: 0.1693883	total: 3m 30s	remaining: 4m 40s
75:	learn: 0.1687128	total: 3m 33s	remaining: 4m 37s
76:	learn: 0.1663447	total: 3m 36s	remaining: 4m 35s
77:	learn: 0.1653100	total: 3m 39s	remaining: 4m 33s
78:	learn: 0.1639088	total: 3m 42s	remaining: 4m 30s
79:	learn: 0.1626474	total: 3m 45s	remaining: 4m 27s
80:	learn: 0.1611973	total: 3m 48s	remaining: 4m 24s
81:	learn: 0.1610886	total: 3m 48s	remaining: 4m 18s
82:	learn: 0.1596615	total: 3m 51s	remaining: 4m 16s
83:	learn: 0.1579888	total: 3m 53s	remaining: 4m 13s
84:	learn: 0.1569754	total: 3m 56s	remaining: 4m 10s
85:	learn: 0.1556031	total: 3m 59s	remaining: 4m 7s
86:	learn: 0.1543729	total: 4m 2s	remaining: 4m 5s
87:	learn: 0.1535340	total: 4m 5s	remaining: 4m 2s
88:	learn: 0.1521182	total: 4m 8s	remaining: 3m 59s

51:	learn: 0.2117694	total: 2m 25s	remaining: 5m 45s
52:	learn: 0.2098150	total: 2m 28s	remaining: 5m 42s
53:	learn: 0.2080135	total: 2m 31s	remaining: 5m 39s
54:	learn: 0.2065806	total: 2m 34s	remaining: 5m 37s
55:	learn: 0.2050026	total: 2m 37s	remaining: 5m 34s
56:	learn: 0.2027799	total: 2m 40s	remaining: 5m 32s
57:	learn: 0.2008854	total: 2m 43s	remaining: 5m 29s
58:	learn: 0.1994558	total: 2m 46s	remaining: 5m 26s
59:	learn: 0.1985382	total: 2m 48s	remaining: 5m 23s
60:	learn: 0.1973229	total: 2m 51s	remaining: 5m 19s
61:	learn: 0.1955668	total: 2m 53s	remaining: 5m 16s
62:	learn: 0.1934756	total: 2m 56s	remaining: 5m 14s
63:	learn: 0.1926862	total: 2m 59s	remaining: 5m 11s
64:	learn: 0.1907545	total: 3m 2s	remaining: 5m 8s
65:	learn: 0.1894429	total: 3m 5s	remaining: 5m 5s
66:	learn: 0.1872921	total: 3m 7s	remaining: 5m 2s
67:	learn: 0.1853329	total: 3m 10s	remaining: 4m 59s
68:	learn: 0.1842162	total: 3m 13s	remaining: 4m 56s
69:	learn: 0.1829773	total: 3m 16s	remaining: 4m 54s

31:	learn: 0.2743351	total: 1m 30s	remaining: 6m 42s
32:	learn: 0.2714230	total: 1m 32s	remaining: 6m 39s
33:	learn: 0.2686625	total: 1m 35s	remaining: 6m 36s
34:	learn: 0.2659265	total: 1m 37s	remaining: 6m 31s
35:	learn: 0.2630418	total: 1m 40s	remaining: 6m 27s
36:	learn: 0.2595740	total: 1m 42s	remaining: 6m 23s
37:	learn: 0.2565211	total: 1m 45s	remaining: 6m 20s
38:	learn: 0.2518820	total: 1m 48s	remaining: 6m 18s
39:	learn: 0.2490447	total: 1m 51s	remaining: 6m 15s
40:	learn: 0.2464448	total: 1m 53s	remaining: 6m 12s
41:	learn: 0.2430201	total: 1m 56s	remaining: 6m 9s
42:	learn: 0.2413001	total: 1m 59s	remaining: 6m 6s
43:	learn: 0.2390793	total: 2m 2s	remaining: 6m 4s
44:	learn: 0.2365301	total: 2m 4s	remaining: 6m
45:	learn: 0.2339975	total: 2m 8s	remaining: 5m 59s
46:	learn: 0.2316639	total: 2m 11s	remaining: 5m 57s
47:	learn: 0.2286327	total: 2m 13s	remaining: 5m 53s
48:	learn: 0.2271376	total: 2m 16s	remaining: 5m 50s
49:	learn: 0.2253403	total: 2m 19s	remaining: 5m 49s
50:

11:	learn: 0.3810042	total: 34.6s	remaining: 7m 50s
12:	learn: 0.3735223	total: 37.2s	remaining: 7m 43s
13:	learn: 0.3655563	total: 40s	remaining: 7m 39s
14:	learn: 0.3580922	total: 42.8s	remaining: 7m 36s
15:	learn: 0.3514017	total: 45.6s	remaining: 7m 33s
16:	learn: 0.3424471	total: 48.2s	remaining: 7m 27s
17:	learn: 0.3361074	total: 50.9s	remaining: 7m 23s
18:	learn: 0.3277905	total: 53.7s	remaining: 7m 21s
19:	learn: 0.3227775	total: 56.4s	remaining: 7m 17s
20:	learn: 0.3185956	total: 59.2s	remaining: 7m 14s
21:	learn: 0.3145669	total: 1m 1s	remaining: 7m 9s
22:	learn: 0.3095836	total: 1m 4s	remaining: 7m 7s
23:	learn: 0.3041085	total: 1m 7s	remaining: 7m 1s
24:	learn: 0.2997686	total: 1m 10s	remaining: 7m
25:	learn: 0.2956254	total: 1m 12s	remaining: 6m 58s
26:	learn: 0.2888168	total: 1m 16s	remaining: 6m 56s
27:	learn: 0.2856539	total: 1m 18s	remaining: 6m 51s
28:	learn: 0.2824554	total: 1m 21s	remaining: 6m 49s
29:	learn: 0.2793080	total: 1m 24s	remaining: 6m 47s
30:	learn: 0.27

167:	learn: 0.1170438	total: 7m 46s	remaining: 19.4s
168:	learn: 0.1168810	total: 7m 49s	remaining: 16.7s
169:	learn: 0.1152587	total: 7m 51s	remaining: 13.9s
170:	learn: 0.1149866	total: 7m 54s	remaining: 11.1s
171:	learn: 0.1149787	total: 7m 56s	remaining: 8.32s
172:	learn: 0.1149505	total: 7m 59s	remaining: 5.54s
173:	learn: 0.1148498	total: 8m 2s	remaining: 2.77s
174:	learn: 0.1148187	total: 8m 4s	remaining: 0us
    fold  4:  [0.90550000]
    ----
    MEAN:     [0.90390000] + [0.00507346]
    FULL:     [0.90390000]



In [26]:
model = lgbm

In [27]:
model = model.fit(S_train, y_train)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


In [28]:
y_pred = model.predict(S_test)

In [29]:
# print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

In [30]:
# confusion_matrix(y_test, y_pred)

In [31]:
y_pred

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [32]:
y_pred.shape

(5000,)

In [33]:
sub = pd.read_csv('submission_ex.csv', encoding='euc-kr')
sub['정답지'] = y_pred
sub.to_csv('sub_stack.csv', index = 0, encoding='euc-kr')

## sub_stack, sub_rf 비교

In [50]:
compare = pd.DataFrame(columns=['status'])

In [51]:
compare

,status


In [52]:
sub_stack = pd.read_csv('sub_stack.csv', encoding='euc-kr')

In [53]:
sub_rf = pd.read_csv('rf_sub.csv', encoding='euc-kr')

In [54]:
sub_stack

,분석대상,정답지
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1
...,...,...
4995,4996,1
4996,4997,1
4997,4998,0
4998,4999,1


In [55]:
sub_rf

,분석대상,정답지
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
4995,4996,1
4996,4997,1
4997,4998,0
4998,4999,1


In [57]:
for i in range(5000):
    if sub_rf['정답지'][i] == sub_stack['정답지'][i]:
        compare.loc[i] = 0
    else:
        compare.loc[i] = 1

In [63]:
(compare['status']==0).sum()

4513

In [64]:
(compare['status']==1).sum()

487